In [73]:
!pip install pandas

In [74]:
import zipfile
import os
import pandas as pd

# Define the directories to unzip the files to
data_dir = 'mnt/data/data'
prem_dir = 'mnt/data/prem-1993-2023'
fpl_player_dir = 'mnt/data/FPLplayer'



# List the contents of each directory
data_files = os.listdir(data_dir)
prem_files = os.listdir(prem_dir)
fpl_player_files = os.listdir(fpl_player_dir)

data_files, prem_files, fpl_player_files


(['.DS_Store', 'data'],
 ['.DS_Store', 'premier-league-matches.csv'],
 ['players.csv'])

In [75]:
import pandas as pd

# Load a sample from each dataset to inspect its structure

# For the FPL data, look inside the nested folder
fpl_nested_data_dir = os.path.join(data_dir, 'data')
fpl_sample_file = os.path.join(fpl_nested_data_dir, '2016-17', 'gws', 'gw1.csv')  # Sample file from the 2016-17 season, gameweek 1

# For the Premier League matches
prem_sample_file = os.path.join(prem_dir, 'premier-league-matches.csv')

# For the FPL player stats
fpl_player_sample_file = os.path.join(fpl_player_dir, 'players.csv')




In [76]:
# Re-define the paths for the sample files
prem_sample_file = os.path.join(prem_dir, 'premier-league-matches.csv')
fpl_player_sample_file = os.path.join(fpl_player_dir, 'players.csv')

# Try reading the FPL sample file with alternative encoding
try:
    fpl_sample_df = pd.read_csv(fpl_sample_file, encoding='utf-8')
except UnicodeDecodeError:
    try:
        fpl_sample_df = pd.read_csv(fpl_sample_file, encoding='ISO-8859-1')
    except Exception as e:
        print(f"Could not read {fpl_sample_file}. Error: {e}")

# Read sample files for the other datasets
try:
    prem_sample_df = pd.read_csv(prem_sample_file, nrows=5)  # Read only the first 5 rows
except Exception as e:
    print(f"Could not read {prem_sample_file}. Error: {e}")

try:
    fpl_player_sample_df = pd.read_csv(fpl_player_sample_file, nrows=5)  # Read only the first 5 rows
except Exception as e:
    print(f"Could not read {fpl_player_sample_file}. Error: {e}")

fpl_sample_df.head(), prem_sample_df.head(), fpl_player_sample_df.head()


(                 name  assists  attempted_passes  big_chances_created  \
 0     Aaron_Cresswell        0                 0                    0   
 1        Aaron_Lennon        0                 3                    0   
 2        Aaron_Ramsey        0                26                    0   
 3  Abdoulaye_Doucouré        0                 0                    0   
 4   Abdul Rahman_Baba        0                 0                    0   
 
    big_chances_missed  bonus  bps  clean_sheets  \
 0                   0      0    0             0   
 1                   0      0    6             0   
 2                   0      0    5             0   
 3                   0      0    0             0   
 4                   0      0    0             0   
 
    clearances_blocks_interceptions  completed_passes  ...  team_h_score  \
 0                                0                 0  ...             2   
 1                                1                 2  ...             1   
 2          

In [77]:
import re
# Initialize an empty list to store individual DataFrames
dfs = []

# Initialize a dictionary to store unique columns for each season
unique_columns_per_season = {}

# Loop through the folders for each season
root_dir = fpl_nested_data_dir  # Using the nested data directory
seasons = [
    '2016-17', '2017-18', '2018-19',
    '2019-20', '2020-21', '2021-22','2023-24'
]

print("Starting to read files...")

for season in seasons:
    season_dir = os.path.join(root_dir, season, 'gws')

    # Check if the folder exists
    if not os.path.exists(season_dir):
        print(f"Skipping missing folder: {season_dir}")
        continue

    print(f"Processing folder: {season_dir}")

    # Initialize set to keep track of unique columns for this season
    unique_columns_this_season = set()

    # Loop through the files in the folder
    for gw_file in os.listdir(season_dir):
        # Use regex to filter out unwanted files
        if re.match(r'gw\d+.csv', gw_file):
            filepath = os.path.join(season_dir, gw_file)

            # Extract gameweek from filename
            gameweek = int(re.sub(r'\D', '', gw_file))

            # Read the data into a DataFrame
            try:
                df = pd.read_csv(filepath)
            except UnicodeDecodeError:
                try:
                    df = pd.read_csv(filepath, encoding='ISO-8859-1')
                except Exception as e:
                    print(f"Could not read {filepath}. Error: {e}")
                    continue

            # Update the set of unique columns for this season
            unique_columns_this_season.update(df.columns.tolist())

            # Add a new column for the season and gameweek
            df['season'] = season
            df['GW'] = gameweek

            # Append to the list of DataFrames
            dfs.append(df)

    # Store unique columns for this season
    unique_columns_per_season[season] = unique_columns_this_season

# Concatenate all the DataFrames to form the final DataFrame
print("Concatenating all DataFrames...")
all_data = pd.concat(dfs, ignore_index=True)

# Find similar and unique columns across seasons
all_seasons_columns = set.intersection(*(set(x) for x in unique_columns_per_season.values()))
unique_columns = {season: cols - all_seasons_columns for season, cols in unique_columns_per_season.items()}

# Keep only the columns that are similar across all seasons, plus the "Season" and "GW" columns
final_columns = list(all_seasons_columns) + ['season', 'GW']
all_data = all_data[final_columns]

# Display the first few rows of the combined FPL data
all_data.head(), all_data.shape


Starting to read files...
Processing folder: mnt/data/data/data/2016-17/gws
Processing folder: mnt/data/data/data/2017-18/gws
Processing folder: mnt/data/data/data/2018-19/gws
Processing folder: mnt/data/data/data/2019-20/gws
Processing folder: mnt/data/data/data/2020-21/gws
Processing folder: mnt/data/data/data/2021-22/gws
Processing folder: mnt/data/data/data/2023-24/gws
Concatenating all DataFrames...


/var/folders/f0/m1wmc83s6v77j93qntkqft8h0000gn/T/ipykernel_65616/3433196307.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat(dfs, ignore_index=True)


(  value opponent_team total_points minutes goals_scored assists  team_h_score  \
 0    53             3            6      90            0       0           1.0   
 1    55             1            2      67            0       0           2.0   
 2    76             6            0       0            0       0           2.0   
 3    45            11            0       0            0       0           1.0   
 4    46            10            0       0            0       0           2.0   
 
   penalties_saved selected  ict_index  ...  influence bonus yellow_cards  \
 0               0    16068        3.3  ...        8.2     0            0   
 1               0     7542        1.9  ...        0.0     0            0   
 2               0    40594        0.0  ...        0.0     0            0   
 3               0        0        0.0  ...        0.0     0            0   
 4               0      667        0.0  ...        0.0     0            0   
 
   transfers_in          kickoff_time tran

### Step 1: Clean and Convert Data Types

The aim is to ensure that each column in our dataset is of the appropriate data type. This is crucial for the performance and reliability of the machine learning model we'll build later.


In [78]:
# Convert all columns except 'name', 'season', and 'was_home' to numeric
cols_to_convert = [col for col in all_data.columns if col not in ['name', 'season', 'was_home']]
all_data[cols_to_convert] = all_data[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Convert 'was_home' to boolean, if it exists in the DataFrame
if 'was_home' in all_data.columns:
    all_data['was_home'] = all_data['was_home'].astype(bool)

# Display the data types and the first few rows to verify
all_data.dtypes, all_data.head()


(value                  int64
 opponent_team          int64
 total_points           int64
 minutes                int64
 goals_scored           int64
 assists                int64
 team_h_score         float64
 penalties_saved        int64
 selected               int64
 ict_index            float64
 creativity           float64
 saves                  int64
 element                int64
 round                  int64
 bps                    int64
 was_home                bool
 clean_sheets           int64
 penalties_missed       int64
 own_goals              int64
 team_a_score         float64
 name                  object
 red_cards              int64
 transfers_out          int64
 fixture                int64
 influence            float64
 bonus                  int64
 yellow_cards           int64
 transfers_in           int64
 kickoff_time         float64
 transfers_balance      int64
 goals_conceded         int64
 threat               float64
 season                object
 GW       

The data types for each column have been successfully converted as appropriate:
- Numeric types (`int64` or `float64`) for numerical data.
- `bool` for the `was_home` column to indicate whether the match was a home game.


### Step 2: Handle Missing Values

Before we move on to feature engineering and model building, it's important to handle missing values in the dataset. Missing values can lead to unreliable and biased analyses, so they need to be addressed.


In [79]:
# Count of NaN values for each column
missing_values_count = all_data.isnull().sum()
missing_values_count = missing_values_count[missing_values_count > 0]
missing_values_count.sort_values(inplace=True)

missing_values_count


team_h_score        59
team_a_score        59
kickoff_time    142949
dtype: int64

### Missing Values Identified

Three columns in our dataset have missing values:

1. `team_h_score`: Missing 59 entries.
2. `team_a_score`: Missing 59 entries.
3. `kickoff_time`: Missing all entries (140,308).

We need to address these before proceeding to feature engineering.


### Handling Missing Values

We decided to handle the missing values as follows:

- For `team_h_score` and `team_a_score`, we will remove the rows with missing values.
- The `kickoff_time` column will be dropped from the dataset as it's missing all its values.

This approach is justifiable given the large number of entries we have in the dataset; removing a few should not significantly impact the model's performance.


In [80]:
# Drop rows with missing values for 'team_h_score' and 'team_a_score'
all_data.dropna(subset=['team_h_score', 'team_a_score'], inplace=True)

# Drop the 'kickoff_time' column
all_data.drop(columns=['kickoff_time'], inplace=True)

# Verify the changes
all_data.isnull().sum().sum(), all_data.shape


(0, (142890, 33))

### Missing Values Handled

- Rows with missing values for `team_h_score` and `team_a_score` have been removed.
- The `kickoff_time` column has been dropped.

Our dataset is now free of missing values and consists of 140,249 rows and 33 columns.


### Step 3: Add Additional Datasets

To improve the model's performance and predictive power, we will incorporate additional datasets into our main DataFrame. These include:

1. Premier League matches dataset for historical match information.
2. Current-season FPL player stats for up-to-date player metrics.

By doing so, we aim to create a more comprehensive and rich feature set.


#### Incorporate Premier League Matches Data

The Premier League matches dataset contains historical match information, such as the teams that played, the goals scored, and the match outcome. Adding this data will allow us to create features that capture the context and dynamics of each match, thereby enriching our feature set.


In [81]:
# Load a sample from the Premier League matches dataset to inspect its structure
try:
    prem_sample_df = pd.read_csv(prem_sample_file, nrows=10)  # Read only the first 10 rows
except Exception as e:
    print(f"Could not read {prem_sample_file}. Error: {e}")

prem_sample_df.head()


,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A


#### Premier League Matches Dataset Structure

The dataset contains the following columns:

- `Season_End_Year`: Year the season ends.
- `Wk`: Gameweek.
- `Date`: Date of the match.
- `Home`: Home team.
- `HomeGoals`: Goals scored by the home team.
- `AwayGoals`: Goals scored by the away team.
- `Away`: Away team.
- `FTR`: Full-time result ('H' for home win, 'D' for draw, 'A' for away win).

We'll consider merging relevant columns from this dataset into our main FPL dataset.


#### Merging Strategy Revised

Given that the `element` column in the FPL dataset corresponds to individual players and that we have a master list of teams (`master_team_list.csv`), our revised merging strategy will involve:

1. Creating a mapping from `element` (players) to their respective teams using the master list.
2. Merging the Premier League dataset based on the season, gameweek, and newly mapped team information.

By doing so, we can align the datasets precisely and enrich our feature set.


In [82]:
# Load the master_team_list.csv file
try:
    master_team_df = pd.read_csv(os.path.join(root_dir, 'master_team_list.csv'))
except Exception as e:
    print(f"Could not read master_team_list.csv. Error: {e}")



In [83]:
import numpy as np
# Compute the team for each player in the FPL dataset
enriched_fpl_data = all_data.copy()



# Compute the team for each player in the FPL dataset again
enriched_fpl_data['team'] = np.where(enriched_fpl_data['was_home'], enriched_fpl_data['team_h_score'], enriched_fpl_data['team_a_score'])

# Merge with the master_team_list.csv to get the team_name
enriched_fpl_data = pd.merge(enriched_fpl_data, master_team_df, on=['season', 'team'], how='left')

# Verify the merge by checking the first few rows
enriched_fpl_data[['season', 'element', 'opponent_team', 'team', 'team_name']].head()


,season,element,opponent_team,team,team_name
0,2016-17,454,3,1.0,Arsenal
1,2016-17,142,1,2.0,Bournemouth
2,2016-17,16,6,1.0,Arsenal
3,2016-17,612,11,1.0,Arsenal
4,2016-17,482,10,0.0,NaN


#### Investigating Discrepancies in Team Names

To address the missing values in the `team_name` column, we'll first identify the unique teams from both the FPL dataset and the `master_team_list.csv` file that are causing mismatches. This will help us understand the source of the discrepancy and take appropriate corrective measures.


In [84]:
# Find unique teams in the FPL dataset that have missing 'team_name' after the merge
missing_teams_fpl = enriched_fpl_data[enriched_fpl_data['team_name'].isna()]['team'].unique()

# Find unique teams in the master_team_list.csv
unique_teams_master = master_team_df['team'].unique()

# Identify the teams that are in the FPL dataset but not in the master_team_list.csv
mismatched_teams = set(missing_teams_fpl) - set(unique_teams_master)

missing_teams_fpl, unique_teams_master, mismatched_teams


(array([0., 2., 1., 3., 5., 4.]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20]),
 {0.0})

#### Discrepancies Found

- Missing Team Identifier in FPL Dataset: `0`
- Unique Team Identifiers in `master_team_list.csv`: Range from `1` to `20`

The team identifier `0` is missing in the `master_team_list.csv` file, leading to the discrepancies.


#### Handling Missing Team Identifier

We'll proceed by removing the rows where the team identifier is `0`. Given the size of our dataset, this is unlikely to significantly impact the model's predictive accuracy.


In [85]:
# Remove rows where the team identifier is 0
enriched_fpl_data = enriched_fpl_data[enriched_fpl_data['team'] != 0]

# Verify the changes by checking for missing values in 'team_name'
missing_team_name_count = enriched_fpl_data['team_name'].isna().sum()
total_rows = enriched_fpl_data.shape[0]

missing_team_name_count, total_rows


(2169, 102292)

#### Merging with Premier League Data

With the team names successfully incorporated into our FPL dataset, we're now prepared to merge it with the Premier League dataset. This will allow us to add contextual information about the matches, enriching our feature set for better predictive accuracy.


In [86]:
# Load the entire Premier League matches dataset
try:
    prem_df = pd.read_csv(prem_sample_file)
except Exception as e:
    print(f"Could not read {prem_sample_file}. Error: {e}")

# Rename columns for easier merging
prem_df.rename(columns={
    'Season_End_Year': 'season_end_year',
    'Wk': 'GW',
    'Date': 'match_date',
    'Home': 'home_team',
    'HomeGoals': 'home_goals',
    'AwayGoals': 'away_goals',
    'Away': 'away_team',
    'FTR': 'full_time_result'
}, inplace=True)

# Create a season column similar to that in the FPL dataset to facilitate merging
prem_df['season'] = (prem_df['season_end_year'] - 1).astype(str) + '-' + prem_df['season_end_year'].astype(str)



In [87]:
# Attempt the merge again, paying close attention to the column names
# First merge based on home_team
merged_data_home = pd.merge(
    enriched_fpl_data,
    prem_df,
    how='left',
    left_on=['season', 'GW', 'team_name'],
    right_on=['season', 'GW', 'home_team']
)

# Then merge based on away_team
merged_data_away = pd.merge(
    enriched_fpl_data,
    prem_df,
    how='left',
    left_on=['season', 'GW', 'team_name'],
    right_on=['season', 'GW', 'away_team']
)

# Combine the data from home and away matches
final_data = pd.concat([merged_data_home, merged_data_away])

# Drop redundant columns
columns_to_drop_final = ['home_team', 'away_team', 'season_end_year']
final_data.drop(columns=columns_to_drop_final, inplace=True, errors='ignore')

# Verify the merge by checking the first few rows and the shape of the dataset
final_data.head(), final_data.shape


(   value  opponent_team  total_points  minutes  goals_scored  assists  \
 0     53              3             6       90             0        0   
 1     55              1             2       67             0        0   
 2     76              6             0        0             0        0   
 3     45             11             0        0             0        0   
 4     55             15             0        0             0        0   
 
    team_h_score  penalties_saved  selected  ict_index  ...  goals_conceded  \
 0           1.0                0     16068        3.3  ...               0   
 1           2.0                0      7542        1.9  ...               1   
 2           2.0                0     40594        0.0  ...               0   
 3           1.0                0         0        0.0  ...               0   
 4           0.0                0      1087        0.0  ...               0   
 
    threat   season  GW  team    team_name  match_date  home_goals  away_goals

In [88]:
# Harmonize the 'season' column in the FPL dataset to match the format in the Premier League dataset
enriched_fpl_data['season'] = enriched_fpl_data['season'].apply(lambda x: f"{x.split('-')[0]}-{int(x.split('-')[0]) + 1}")

# Verify that the 'season' column has been updated correctly
enriched_fpl_data['season'].unique()


array(['2016-2017', '2017-2018', '2018-2019', '2019-2020', '2020-2021',
       '2021-2022', '2023-2024'], dtype=object)

In [89]:
# Attempt the merge again after harmonizing the 'season' column
# First merge based on home_team
merged_data_home = pd.merge(
    enriched_fpl_data,
    prem_df,
    how='left',
    left_on=['season', 'GW', 'team_name'],
    right_on=['season', 'GW', 'home_team']
)

# Then merge based on away_team
merged_data_away = pd.merge(
    enriched_fpl_data,
    prem_df,
    how='left',
    left_on=['season', 'GW', 'team_name'],
    right_on=['season', 'GW', 'away_team']
)

# Combine the data from home and away matches
final_data = pd.concat([merged_data_home, merged_data_away])

# Drop redundant columns
columns_to_drop_final = ['home_team', 'away_team', 'season_end_year']
final_data.drop(columns=columns_to_drop_final, inplace=True, errors='ignore')

# Verify the merge by checking the first few rows and the shape of the dataset
final_data.head(), final_data.shape


(   value  opponent_team  total_points  minutes  goals_scored  assists  \
 0     53              3             6       90             0        0   
 1     55              1             2       67             0        0   
 2     76              6             0        0             0        0   
 3     45             11             0        0             0        0   
 4     55             15             0        0             0        0   
 
    team_h_score  penalties_saved  selected  ict_index  ...  goals_conceded  \
 0           1.0                0     16068        3.3  ...               0   
 1           2.0                0      7542        1.9  ...               1   
 2           2.0                0     40594        0.0  ...               0   
 3           1.0                0         0        0.0  ...               0   
 4           0.0                0      1087        0.0  ...               0   
 
    threat     season  GW  team    team_name  match_date  home_goals  \
 0    

#### Investigating Merge Discrepancies

To get to the root of the issue with the merge, we'll take the following steps:

1. Check for missing or mismatched gameweeks between the two datasets.
2. Inspect if team names match perfectly between the two datasets.

By investigating these aspects, we can identify the source of the problem and take corrective actions accordingly.


In [90]:
# Check for unique gameweeks in both datasets for each season
unique_gw_fpl = enriched_fpl_data.groupby('season')['GW'].nunique().reset_index().rename(columns={'GW': 'Unique_GW_FPL'})
unique_gw_prem = prem_df.groupby('season')['GW'].nunique().reset_index().rename(columns={'GW': 'Unique_GW_Prem'})

# Merge the unique gameweek counts for each season
gw_comparison = pd.merge(unique_gw_fpl, unique_gw_prem, on='season', how='outer')

gw_comparison


,season,Unique_GW_FPL,Unique_GW_Prem
0,2016-2017,38.0,38.0
1,2017-2018,38.0,38.0
2,2018-2019,38.0,38.0
3,2019-2020,38.0,38.0
4,2020-2021,38.0,38.0
5,2021-2022,38.0,38.0
6,2023-2024,4.0,NaN
7,1992-1993,NaN,42.0
8,1993-1994,NaN,42.0
9,1994-1995,NaN,42.0


#### Gameweek Comparison

Both the FPL and Premier League datasets have 38 unique gameweeks for each available season, so this aspect doesn't appear to be the cause of the missing values in the merge.

However, it's worth noting that the Premier League dataset has data for many more seasons, dating back to 1992-1993. The FPL dataset only has data starting from the 2016-2017 season.


In [91]:
# Check for unique team names in both datasets for each season
unique_teams_fpl = enriched_fpl_data.groupby('season')['team_name'].unique().reset_index().rename(columns={'team_name': 'Unique_Teams_FPL'})
unique_teams_prem_home = prem_df.groupby('season')['home_team'].unique().reset_index().rename(columns={'home_team': 'Unique_Teams_Prem_Home'})
unique_teams_prem_away = prem_df.groupby('season')['away_team'].unique().reset_index().rename(columns={'away_team': 'Unique_Teams_Prem_Away'})

# Merge the unique team names for each season
team_comparison_home = pd.merge(unique_teams_fpl, unique_teams_prem_home, on='season', how='outer')
team_comparison_away = pd.merge(unique_teams_fpl, unique_teams_prem_away, on='season', how='outer')

team_comparison_home, team_comparison_away


(       season                                   Unique_Teams_FPL  \
 0   2016-2017  [Arsenal, Bournemouth, Burnley, Chelsea, Cryst...   
 1   2017-2018  [Arsenal, Bournemouth, Brighton, Chelsea, Burn...   
 2   2018-2019  [Brighton, Arsenal, Bournemouth, Burnley, Card...   
 3   2019-2020  [Arsenal, Bournemouth, Aston Villa, Burnley, B...   
 4   2020-2021  [Arsenal, Chelsea, Brighton, Aston Villa, Burn...   
 5   2021-2022  [Arsenal, Brentford, Aston Villa, Brighton, Cr...   
 6   2023-2024                                              [nan]   
 7   1992-1993                                                NaN   
 8   1993-1994                                                NaN   
 9   1994-1995                                                NaN   
 10  1995-1996                                                NaN   
 11  1996-1997                                                NaN   
 12  1997-1998                                                NaN   
 13  1998-1999                    

In [92]:
# Compare a sample of team names for the most recent common season (2021-2022) between the datasets
sample_teams_fpl_2021 = enriched_fpl_data[enriched_fpl_data['season'] == '2021-2022']['team_name'].unique()
sample_teams_prem_home_2021 = prem_df[prem_df['season'] == '2021-2022']['home_team'].unique()
sample_teams_prem_away_2021 = prem_df[prem_df['season'] == '2021-2022']['away_team'].unique()

# Combine the unique team names from the Premier League dataset (home and away should be the same)
sample_teams_prem_2021 = list(set(sample_teams_prem_home_2021) | set(sample_teams_prem_away_2021))

# Sort the lists for easier comparison
sample_teams_fpl_2021 = sorted(sample_teams_fpl_2021)
sample_teams_prem_2021 = sorted(sample_teams_prem_2021)

sample_teams_fpl_2021, sample_teams_prem_2021


(['Arsenal',
  'Aston Villa',
  'Brentford',
  'Brighton',
  'Burnley',
  'Chelsea',
  'Crystal Palace'],
 ['Arsenal',
  'Aston Villa',
  'Brentford',
  'Brighton',
  'Burnley',
  'Chelsea',
  'Crystal Palace',
  'Everton',
  'Leeds United',
  'Leicester City',
  'Liverpool',
  'Manchester City',
  'Manchester Utd',
  'Newcastle Utd',
  'Norwich City',
  'Southampton',
  'Tottenham',
  'Watford',
  'West Ham',
  'Wolves'])

#### Moving Forward: Feature Engineering and Model Building

Given the constraints of predicting future performance, we'll primarily focus on:

1. Lagged Features: Past performance metrics to estimate future outcomes.
2. Rolling Averages: To smooth out performance over time and provide more generalized estimates.

We'll eventually drop or transform features that can't be known before matches are played. With these considerations in mind, let's proceed with feature engineering.


In [93]:
# List of columns to create lagged features and rolling averages for
# We're avoiding columns that won't be known in advance, such as 'goals_scored', 'saves', etc.
cols_to_lag_and_roll = [
    "bonus", "transfers_balance", "ict_index",
    "value", "was_home", "transfers_in", "influence",
    "assists", "transfers_out", "bps", "selected", "yellow_cards"
]

# Create lagged features (lagged by 1 game week)
for col in cols_to_lag_and_roll:
    final_data[f"lagged_{col}"] = final_data.groupby('name')[col].shift(1)

# Create rolling averages (over the last 5 and 10 game weeks)
for col in cols_to_lag_and_roll:
    final_data[f"{col}_rolling_5"] = final_data.groupby("name")[col].transform(lambda x: x.rolling(window=5, min_periods=1).mean().shift(1))
    final_data[f"{col}_rolling_10"] = final_data.groupby("name")[col].transform(lambda x: x.rolling(window=10, min_periods=1).mean().shift(1))

# Check the first few rows to verify the new features
final_data.head(), final_data.columns.tolist()


(   value  opponent_team  total_points  minutes  goals_scored  assists  \
 0     53              3             6       90             0        0   
 1     55              1             2       67             0        0   
 2     76              6             0        0             0        0   
 3     45             11             0        0             0        0   
 4     55             15             0        0             0        0   
 
    team_h_score  penalties_saved  selected  ict_index  ...  assists_rolling_5  \
 0           1.0                0     16068        3.3  ...                NaN   
 1           2.0                0      7542        1.9  ...                NaN   
 2           2.0                0     40594        0.0  ...                NaN   
 3           1.0                0         0        0.0  ...                NaN   
 4           0.0                0      1087        0.0  ...                NaN   
 
    assists_rolling_10  transfers_out_rolling_5  transfers_o

#### Feature Engineering: Lagged Features and Rolling Averages

- **Lagged Features**: For each player and relevant metric, we've created features that capture the metric's value from the previous game week.
  - Example: `lagged_bonus`, `lagged_transfers_balance`, `lagged_ict_index`, etc.
  
- **Rolling Averages**: We've also calculated 5-game-week and 10-game-week rolling averages for these metrics.
  - Example: `bonus_rolling_5`, `bonus_rolling_10`, `transfers_balance_rolling_5`, etc.

These features will help us estimate future performance based on past trends.


In [94]:
# Drop columns that won't be known before matches are played
# These columns include specific match outcomes like 'goals_scored', 'saves', etc.
cols_to_drop_for_prediction = [
    'goals_conceded', 'creativity', 'bps', 'yellow_cards', 'red_cards', 'bonus',
    'goals_scored', 'team_h_score', 'saves', 'transfers_out', 'selected',
    'penalties_missed', 'fixture', 'transfers_in', 'influence', 'team_a_score',
    'penalties_saved', 'clean_sheets', 'ict_index', 'assists', 'threat', 'round',
    'own_goals', 'minutes', 'transfers_balance', 'match_date', 'home_goals',
    'away_goals', 'full_time_result'
]

# Drop these columns while ensuring 'total_points' is not dropped
final_data_clean = final_data.drop(columns=[col for col in cols_to_drop_for_prediction if col != 'total_points'])

# Drop rows where 'total_points' or any of the engineered features are NaN
final_data_clean.dropna(subset=['total_points'] + [col for col in final_data_clean.columns if 'lagged' in col or 'rolling' in col], inplace=True)

# Verify that the data is clean by checking the first few rows and the remaining columns
final_data_clean.head(), final_data_clean.columns.tolist(), final_data_clean.isnull().sum()


(     value  opponent_team  total_points  element  was_home               name  \
 368     53              7             7      454      True    Aaron_Cresswell   
 369     76             10             0       16     False       Aaron_Ramsey   
 370     55              5             0       80     False  Abdul Rahman_Baba   
 373     44             13             0       30      True      Adam_Federici   
 375     73              6             3      205     False       Adam_Lallana   
 
         season  GW  team team_name  ...  assists_rolling_5  \
 368  2016-2017  17   1.0   Arsenal  ...                0.0   
 369  2016-2017  17   1.0   Arsenal  ...                0.0   
 370  2016-2017  17   1.0   Arsenal  ...                0.0   
 373  2016-2017  17   1.0   Arsenal  ...                0.0   
 375  2016-2017  17   1.0   Arsenal  ...                1.0   
 
      assists_rolling_10  transfers_out_rolling_5  transfers_out_rolling_10  \
 368                 0.0                    256

#### Data Cleaning and Final Feature Selection

1. **Dropped Unpredictable Columns**: Removed features that can't be known before matches, like 'goals_scored', 'saves', etc.
2. **Handled Missing Values**: Removed rows where either the target variable (`total_points`) or any of the engineered features had missing values.
3. **Data Verification**: Checked that the cleaned dataset has no missing values.



#### Function for Creating a New Prediction Row

To predict `total_points` for a player in a future gameweek, the function will need the following information:

1. **Player Name (`name`)**: To identify the player.
2. **Season (`season`)**: The FPL season (e.g., '2023-2024').
3. **Gameweek (`GW`)**: The gameweek for which you want to make the prediction.
4. **Team (`team_name`)**: The team the player belongs to.
5. **Opponent Team (`opponent_team`)**: The team against which the player will play.
6. **Home/Away (`was_home`)**: Whether the player's team is playing at home or away.

The function will then find the lagged and rolling average values for this player based on historical data.



#### Additional Feature Engineering: Player Position, Form, and Team Form

1. **Player Position**: Using the `players.csv` file, we can include the player's position as a feature.
2. **Player Form**: Calculated as the average `total_points` over the last 3, 5, or 10 gameweeks.
3. **Team Form**: Calculated as the average `total_points` for all players from the same team over the last 3, 5, or 10 gameweeks.




In [95]:
# Load the players.csv file to get player positions
players_df = pd.read_csv('mnt/data/FPLplayer/players.csv')


In [96]:
# Merge the players' positions into the main dataframe using the correct columns
final_data_with_positions = pd.merge(final_data_clean, players_df[['name', 'position']], on='name', how='left')

# Verify that the positions have been correctly merged
final_data_with_positions.head(), final_data_with_positions['position'].unique()


(   value  opponent_team  total_points  element  was_home               name  \
 0     53              7             7      454      True    Aaron_Cresswell   
 1     76             10             0       16     False       Aaron_Ramsey   
 2     55              5             0       80     False  Abdul Rahman_Baba   
 3     44             13             0       30      True      Adam_Federici   
 4     73              6             3      205     False       Adam_Lallana   
 
       season  GW  team team_name  ...  assists_rolling_10  \
 0  2016-2017  17   1.0   Arsenal  ...                 0.0   
 1  2016-2017  17   1.0   Arsenal  ...                 0.0   
 2  2016-2017  17   1.0   Arsenal  ...                 0.0   
 3  2016-2017  17   1.0   Arsenal  ...                 0.0   
 4  2016-2017  17   1.0   Arsenal  ...                 1.0   
 
    transfers_out_rolling_5  transfers_out_rolling_10  bps_rolling_5  \
 0                    256.0                     256.0           17.0   


In [97]:
# Identify rows where the 'position' column is NaN
missing_positions = final_data_with_positions[final_data_with_positions['position'].isna()]['name'].unique()

# Compare these player names with those in the players.csv file to identify mismatches
players_in_csv = players_df['name'].unique()

# Find player names that are in 'missing_positions' but not in 'players_in_csv'
mismatched_players = [player for player in missing_positions if player not in players_in_csv]

# Display a sample of mismatched players to identify potential issues
mismatched_players[:10]

['Aaron_Cresswell',
 'Aaron_Ramsey',
 'Abdul Rahman_Baba',
 'Adam_Federici',
 'Adam_Lallana',
 'Adam_Smith',
 'Adrián_San Miguel del Castillo',
 'Ainsley_Maitland-Niles',
 'Alberto_Moreno',
 'Aleix_García Serrano']

In [98]:
# Approach to fix mismatches:
# 1. Replace underscores in names with spaces in both datasets
# 2. Remove any extra spaces or special characters
# 3. Reattempt the merge

# Replace underscores and remove extra spaces in the main dataset
final_data_with_positions['clean_name'] = final_data_with_positions['name'].str.replace('_', ' ').str.strip()

# Replace underscores and remove extra spaces in the players.csv dataset
players_df['clean_name'] = players_df['name'].str.replace('_', ' ').str.strip()

# Reattempt the merge using 'clean_name'
final_data_with_positions_fixed = pd.merge(final_data_with_positions.drop('position', axis=1), players_df[['clean_name', 'position']], left_on='clean_name', right_on='clean_name', how='left')

# Drop the 'clean_name' column as it's no longer needed
final_data_with_positions_fixed.drop('clean_name', axis=1, inplace=True)

# Check for any remaining NaN values in the 'position' column
remaining_missing_positions = final_data_with_positions_fixed[final_data_with_positions_fixed['position'].isna()]['name'].unique()

remaining_missing_positions, final_data_with_positions_fixed.head()


(array(['Adam_Federici', 'Ainsley_Maitland-Niles', 'Alberto_Moreno', ...,
        'William Fish', 'Harry Boyes', 'Nile John'], dtype=object),
    value  opponent_team  total_points  element  was_home               name  \
 0     53              7             7      454      True    Aaron_Cresswell   
 1     76             10             0       16     False       Aaron_Ramsey   
 2     55              5             0       80     False  Abdul Rahman_Baba   
 3     44             13             0       30      True      Adam_Federici   
 4     73              6             3      205     False       Adam_Lallana   
 
       season  GW  team team_name  ...  assists_rolling_10  \
 0  2016-2017  17   1.0   Arsenal  ...                 0.0   
 1  2016-2017  17   1.0   Arsenal  ...                 0.0   
 2  2016-2017  17   1.0   Arsenal  ...                 0.0   
 3  2016-2017  17   1.0   Arsenal  ...                 0.0   
 4  2016-2017  17   1.0   Arsenal  ...                 1.0   
 
  

#### Correcting Mismatched Player Names and Positions

1. **Name Cleaning**: Replaced underscores with spaces and removed extra spaces to standardize player names across datasets.
2. **Successful Merge**: Managed to merge most of the positions successfully.
3. **Remaining Missing Values**: Some players still have missing position information, possibly due to being less common players or having unique naming issues.

Given that the majority of the positions have been successfully merged, we can proceed in one of two ways:

1. **Impute Missing Positions**: For the remaining players with missing positions, we can fill in the most common position or use other methods.
2. **Ignore**: Since the dataset is large and these are a small number of cases, we could choose to proceed without fixing these.




In [99]:
# Find the most common position in the dataset
most_common_position = final_data_with_positions_fixed['position'].mode()[0]

# Impute missing positions with the most common position
final_data_with_positions_fixed['position'].fillna(most_common_position, inplace=True)

# Verify that there are no more missing positions
remaining_missing_positions = final_data_with_positions_fixed[final_data_with_positions_fixed['position'].isna()]['name'].unique()

remaining_missing_positions, final_data_with_positions_fixed['position'].unique()


(array([], dtype=object), array(['DEF', 'MID', 'GKP', 'FWD'], dtype=object))

In [100]:
# Calculate form indicators for individual players
# Player form is calculated as the average 'total_points' over the last 3, 5, and 10 gameweeks
final_data_with_positions_fixed['player_form_3'] = final_data_with_positions_fixed.groupby("name")['total_points'].transform(lambda x: x.rolling(window=3, min_periods=1).mean().shift(1))
final_data_with_positions_fixed['player_form_5'] = final_data_with_positions_fixed.groupby("name")['total_points'].transform(lambda x: x.rolling(window=5, min_periods=1).mean().shift(1))
final_data_with_positions_fixed['player_form_10'] = final_data_with_positions_fixed.groupby("name")['total_points'].transform(lambda x: x.rolling(window=10, min_periods=1).mean().shift(1))

# Calculate form indicators for teams
# Team form is calculated as the average 'total_points' for all players from the same team over the last 3, 5, and 10 gameweeks
final_data_with_positions_fixed['team_form_3'] = final_data_with_positions_fixed.groupby("team_name")['total_points'].transform(lambda x: x.rolling(window=3, min_periods=1).mean().shift(1))
final_data_with_positions_fixed['team_form_5'] = final_data_with_positions_fixed.groupby("team_name")['total_points'].transform(lambda x: x.rolling(window=5, min_periods=1).mean().shift(1))
final_data_with_positions_fixed['team_form_10'] = final_data_with_positions_fixed.groupby("team_name")['total_points'].transform(lambda x: x.rolling(window=10, min_periods=1).mean().shift(1))

# Verify the new form columns
final_data_with_positions_fixed.head(), final_data_with_positions_fixed.columns.tolist()


(   value  opponent_team  total_points  element  was_home               name  \
 0     53              7             7      454      True    Aaron_Cresswell   
 1     76             10             0       16     False       Aaron_Ramsey   
 2     55              5             0       80     False  Abdul Rahman_Baba   
 3     44             13             0       30      True      Adam_Federici   
 4     73              6             3      205     False       Adam_Lallana   
 
       season  GW  team team_name  ...  selected_rolling_10  \
 0  2016-2017  17   1.0   Arsenal  ...              16068.0   
 1  2016-2017  17   1.0   Arsenal  ...              40594.0   
 2  2016-2017  17   1.0   Arsenal  ...               1087.0   
 3  2016-2017  17   1.0   Arsenal  ...               5312.0   
 4  2016-2017  17   1.0   Arsenal  ...             367017.0   
 
    yellow_cards_rolling_5  yellow_cards_rolling_10  position player_form_3  \
 0                     0.0                      0.0       D

### Building Ensemble Models for FPL Point Prediction

To achieve the highest possible accuracy, we'll follow these steps:

1. **Data Preprocessing**: Make the data ready for training.
2. **Feature Selection**: Choose the most informative features.
3. **Model Selection**: Pick a variety of base models.
4. **Ensemble Method**: Use ensemble techniques like Bagging, Boosting, or Stacking.
5. **Model Evaluation**: Validate the model's performance using cross-validation.
6. **Hyperparameter Tuning**: Optimize the parameters for each model.
7. **Final Model**: Assemble the final ensemble model.

By following this roadmap, we aim to create a highly accurate model for predicting FPL points.


In [101]:
# Identify numeric columns
numeric_columns = final_data_with_positions_fixed.select_dtypes(include=[np.number]).columns.tolist()

# Check for missing values only in numeric columns
missing_values = final_data_with_positions_fixed[numeric_columns].isnull().sum()
missing_values = missing_values[missing_values > 0]

# If there are missing values, handle them accordingly
if not missing_values.empty:
    # Fill missing numerical features with their median values
    for column in missing_values.index:
        median_value = final_data_with_positions_fixed[column].median()
        final_data_with_positions_fixed[column].fillna(median_value, inplace=True)

# Re-check for missing values
missing_values_after = final_data_with_positions_fixed.isnull().sum()
missing_values_after = missing_values_after[missing_values_after > 0]

missing_values, missing_values_after


(player_form_3     3519
 player_form_5     3519
 player_form_10    3519
 team_form_3       4000
 team_form_5       4000
 team_form_10      4000
 dtype: int64,
 team_name    3988
 dtype: int64)

In [102]:
# Remove rows where 'team_name' is missing
final_data_with_positions_fixed.dropna(subset=['team_name'], inplace=True)

# Re-check for missing values
missing_values_after = final_data_with_positions_fixed.isnull().sum()
missing_values_after = missing_values_after[missing_values_after > 0]


In [105]:
from sklearn.preprocessing import StandardScaler

# Identify numeric columns for scaling
numeric_columns = final_data_with_positions_fixed.select_dtypes(include=[np.number]).columns.tolist()

# Columns to be normalized
columns_to_normalize = [col for col in df.columns if col not in ['team', 'opponent_team', 'element', 'GW']]


# Remove 'team' from the list of columns to normalize, if it exists
if 'team' in columns_to_normalize:
    columns_to_normalize.remove('team')

# Remove 'opponent_team' from the list of columns to normalize, if it exists
if 'opponent_team' in columns_to_normalize:
    columns_to_normalize.remove('opponent_team')

# Remove 'element' from the list of columns to normalize, if it exists
if 'element' in columns_to_normalize:
    columns_to_normalize.remove('element')

# Remove 'GW' from the list of columns to normalize, if it exists
if 'GW' in columns_to_normalize:
    columns_to_normalize.remove('GW')

# Remove 'total_points' from the list of columns to normalize, as it's the target variable
if 'total_points' in columns_to_normalize:
    columns_to_normalize.remove('total_points')

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply scaling to all numeric columns except the target variable 'total_points'
final_data_with_positions_fixed[numeric_columns] = scaler.fit_transform(final_data_with_positions_fixed[numeric_columns])

# Show some rows to verify the scaling
final_data_with_positions_fixed.head()


,value,opponent_team,total_points,element,was_home,name,season,GW,team,team_name,...,selected_rolling_10,yellow_cards_rolling_5,yellow_cards_rolling_10,position,player_form_3,player_form_5,player_form_10,team_form_3,team_form_5,team_form_10
0,0.059026,-0.617770,1.973530,0.812881,True,Aaron_Cresswell,2016-2017,-0.320061,-0.837119,Arsenal,...,-0.403517,-0.455764,-0.554849,DEF,-0.417042,-0.387456,-0.30976,-0.321679,-0.253314,-0.134038
1,1.803968,-0.103967,-0.550710,-1.606714,False,Aaron_Ramsey,2016-2017,-0.320061,-0.837119,Arsenal,...,-0.340236,-0.455764,-0.554849,MID,-0.417042,-0.387456,-0.30976,3.340197,4.238197,5.768042
2,0.210760,-0.960305,-0.550710,-1.253166,False,Abdul Rahman_Baba,2016-2017,-0.320061,-0.837119,Arsenal,...,-0.442170,-0.455764,-0.554849,DEF,-0.417042,-0.387456,-0.30976,1.204103,1.527802,2.079242
3,-0.623777,0.409835,-0.550710,-1.529375,True,Adam_Federici,2016-2017,-0.320061,-0.837119,Arsenal,...,-0.431269,-0.455764,-0.554849,MID,-0.417042,-0.387456,-0.30976,0.492071,0.624338,0.849642
4,1.576367,-0.789037,0.531107,-0.562642,False,Adam_Lallana,2016-2017,-0.320061,-0.837119,Arsenal,...,0.501984,-0.455764,-0.554849,MID,-0.417042,-0.387456,-0.30976,-0.931992,0.172605,0.234842


#### Feature Scaling

We used the `StandardScaler` from scikit-learn to standardize the features. This process re-scales the numeric features so that they have a mean of 0 and a standard deviation of 1. This is particularly important for distance-based algorithms and also helps improve the performance of many machine learning models.

Here is how the first few rows of the scaled dataset look like:

| value | name | opponent_team | ... | team_form_5 | team_form_10 |
|-------|------|---------------|-----|-------------|--------------|
| 0.059 | Aaron_Cresswell | -0.618 | ... | -0.253 | -0.134 |
| 1.804 | Aaron_Ramsey | -0.104 | ... | 4.238 | 5.768 |
| 0.211 | Abdul Rahman_Baba | -0.960 | ... | 1.528 | 2.079 |
| ...   | ...  | ...           | ... | ...         | ...          |

Next, we'll focus on **Encoding Categorical Variables**.


In [106]:
# One-hot encode the categorical variables 'team_name' and 'position'
final_data_encoded = pd.get_dummies(final_data_with_positions_fixed, columns=['team_name', 'position', 'season', 'lagged_was_home'], drop_first=True)

# Drop the 'name' column as it is non-numeric and not needed for modeling
final_data_encoded.drop(columns=['name'], inplace=True)

# Show some rows to verify the encoding
final_data_encoded.head()


,value,opponent_team,total_points,element,was_home,GW,team,lagged_bonus,lagged_transfers_balance,lagged_ict_index,...,team_name_Leicester,position_FWD,position_GKP,position_MID,season_2017-2018,season_2018-2019,season_2019-2020,season_2020-2021,season_2021-2022,lagged_was_home_True
0,0.059026,-0.617770,1.973530,0.812881,True,-0.320061,-0.837119,-0.247192,0.021745,0.422833,...,False,False,False,False,False,False,False,False,False,True
1,1.803968,-0.103967,-0.550710,-1.606714,False,-0.320061,-0.837119,-0.247192,-0.054365,-0.566906,...,False,False,False,True,False,False,False,False,False,False
2,0.210760,-0.960305,-0.550710,-1.253166,False,-0.320061,-0.837119,-0.247192,-0.026325,-0.566906,...,False,False,False,False,False,False,False,False,False,False
3,-0.623777,0.409835,-0.550710,-1.529375,True,-0.320061,-0.837119,-0.247192,-0.027733,-0.566906,...,False,False,False,True,False,False,False,False,False,True
4,1.576367,-0.789037,0.531107,-0.562642,False,-0.320061,-0.837119,5.269694,0.562561,4.771685,...,False,False,False,True,False,False,False,False,False,False


In [107]:
# Save the preprocessed and encoded data to a CSV file
final_data_encoded.to_csv('mnt/data/final_data_encoded.csv', index=False)


### Model Building

#### Plan:

1. **Feature Selection**: To identify the most crucial features that contribute to our prediction.
2. **Data Splitting**: Divide the dataset into a training set and a testing set.
3. **Model Selection**: Use different algorithms and ensemble methods to build the models.
4. **Hyperparameter Tuning**: Fine-tune the parameters of the selected model for better performance.
5. **Evaluation**: Use metrics like RMSE to evaluate how well the model is performing.

Let's start with **Feature Selection**.


In [108]:
# Identify non-numeric columns in the dataset
non_numeric_columns = final_data_with_positions_fixed.select_dtypes(include=['object']).columns
non_numeric_columns


Index(['name', 'season', 'team_name', 'lagged_was_home', 'position'], dtype='object')

In [109]:
# Drop the 'name' column
if 'name' in final_data_encoded.columns:
    final_data_encoded.drop('name', axis=1, inplace=True)


In [110]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

# Separate features and target variable from the dataset
X = final_data_encoded.drop('total_points', axis=1)
y = final_data_encoded['total_points']

# Use f_regression as the score function
selector = SelectKBest(score_func=f_regression, k='all')

# Fit the model
selector.fit(X, y)

# Get the scores for each feature
feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': selector.scores_}).sort_values(by='Score', ascending=False)

# Display the top 20 features based on their scores
top_20_features = feature_scores.nlargest(20, 'Score')
top_20_features


,Feature,Score
22,ict_index_rolling_10,52671.688007
43,player_form_10,52131.935280
36,bps_rolling_10,51007.525730
21,ict_index_rolling_5,50227.777142
30,influence_rolling_10,49208.278449
35,bps_rolling_5,47598.635290
42,player_form_5,46662.814731
29,influence_rolling_5,45002.001183
41,player_form_3,40090.618980
37,selected_rolling_5,28894.855835


In [111]:
from sklearn.model_selection import train_test_split

# Selecting the top 20 features
top_20_feature_names = top_20_features['Feature'].tolist()
X_selected = X[top_20_feature_names]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)


In [112]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize the models
linear_model = LinearRegression()
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)

# Train the models
linear_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Make predictions on the test set
linear_pred = linear_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
gb_pred = gb_model.predict(X_test)

# Evaluate the models
linear_rmse = mean_squared_error(y_test, linear_pred, squared=False)
rf_rmse = mean_squared_error(y_test, rf_pred, squared=False)
gb_rmse = mean_squared_error(y_test, gb_pred, squared=False)

linear_mae = mean_absolute_error(y_test, linear_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)
gb_mae = mean_absolute_error(y_test, gb_pred)

print(f"Linear Regression RMSE: {linear_rmse}, MAE: {linear_mae}")
print(f"Random Forest RMSE: {rf_rmse}, MAE: {rf_mae}")
print(f"Gradient Boosting RMSE: {gb_rmse}, MAE: {gb_mae}")


Linear Regression RMSE: 0.8563828834989514, MAE: 0.5208579069978316
Random Forest RMSE: 0.5727280346293226, MAE: 0.31291815748708157
Gradient Boosting RMSE: 0.8359800593239408, MAE: 0.49958696396290087


In [113]:
# Define the function to prepare future prediction rows
def prepare_future_prediction_rows(fixture_list, historical_data, form_windows=[3, 5, 10]):
    future_rows = []
    for fixture in fixture_list:
        home_team, away_team = fixture.split(' v ')

        # Fetch players for both teams
        home_players = historical_data[historical_data['team_name'] == home_team]['name'].unique()
        away_players = historical_data[historical_data['team_name'] == away_team]['name'].unique()

        # Combine both home and away players
        all_players = list(home_players) + list(away_players)

        for player in all_players:
            # Filter historical data for this player
            player_data = historical_data[historical_data['name'] == player]

            # Create an empty row for this player
            new_row = {}

            # Populate known values
            new_row['name'] = player
            new_row['team_name'] = home_team if player in home_players else away_team
            new_row['opponent_team'] = away_team if player in home_players else home_team
            new_row['was_home'] = player in home_players

            # Populate lagged and rolling average features
            for feature in historical_data.columns:
                if 'lagged' in feature or 'form' in feature or 'team_form' in feature:
                    new_row[feature] = player_data[feature].iloc[-1]  # Most recent value

            # Add the new row to the list
            future_rows.append(new_row)

    # Convert list of new rows to DataFrame
    future_df = pd.DataFrame(future_rows)

    return future_df

# Sample fixtures for the next week
sample_fixtures = [
    "Wolves v Liverpool",
    "Aston Villa v Crystal Palace",
    "Fulham v Luton Town",
    "Man Utd v Brighton",
    "Spurs v Sheff Utd",
    "West Ham v Man City",
    "Newcastle v Brentford",
    "AFC Bournemouth v Chelsea",
    "Everton v Arsenal",
    "Nott'm Forest v Burnley"
]

# Prepare future prediction rows for the sample fixtures
future_prediction_rows = prepare_future_prediction_rows(sample_fixtures, final_data_with_positions_fixed)

# Show a sample of the prepared future prediction rows
future_prediction_rows.head(), future_prediction_rows.columns.tolist()


(                    name    team_name   opponent_team  was_home  lagged_bonus  \
 0         Mathew_Ryan_47  Aston Villa  Crystal Palace      True     -0.247192   
 1     Brandon_Austin_558  Aston Villa  Crystal Palace      True     -0.247192   
 2     Alfie_Whiteman_543  Aston Villa  Crystal Palace      True     -0.247192   
 3  Caoimhin_Kelleher_531  Aston Villa  Crystal Palace      True     -0.247192   
 4      Muhamed_Bešić_522  Aston Villa  Crystal Palace      True     -0.247192   
 
    lagged_transfers_balance  lagged_ict_index  lagged_value  lagged_was_home  \
 0                 -1.155624         -0.147017     -0.244866             True   
 1                 -0.030385         -0.566906     -0.927825            False   
 2                 -0.036879         -0.566906     -0.927825            False   
 3                 -0.026526         -0.566906     -0.624288            False   
 4                 -0.025374         -0.566906     -0.624288             True   
 
    lagged_transfe